<a href="https://colab.research.google.com/github/bezland/2023-Tools-V1/blob/main/2023%20Draft%20Rankings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Starter

Install custom packages

In [1]:
!pip install sleeper-api-wrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 1.2 MB/s 
     |████████████████████████████████| 229 kB 9.6 MB/s 
     |████████████████████████████████| 58 kB 3.3 MB/s 
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: idna
    Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pytest
    Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

Import Packages

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import requests
from bs4 import BeautifulSoup
import statistics
import csv
import re
from sleeper_wrapper import League
from sleeper_wrapper import Stats
from sleeper_wrapper import Players
from sleeper_wrapper import User
import string

Sleeper tests

https://pypi.org/project/sleeper-api-wrapper/#players

Initialising API libraries

In [ ]:
league = League(839437223525974016)
stats = Stats()
players = Players()

Master data setup

In [ ]:
week = 14
season = 2022
season_type = ["regular", "pre", "post"]
starting_roster_size = 9

players_master = players.get_all_players()
rosters_master = league.get_rosters()
user_list_master = league.get_users()
weekly_projections = stats.get_week_projections(season_type[0], season, week)
matchups_master = league.get_matchups(week)

rosters_master_temp = [rosters_master[0], rosters_master[1], rosters_master[3], rosters_master [4], rosters_master[6], rosters_master[8], rosters_master[10], rosters_master[11]]

#removal_list = [' III', ' II', ' Jr.', ' Jr', ' St.', ' Sr.', ' V', 'III', 'II', 'Jr.', 'Jr', 'St.', ' St', 'Sr.', ' Sr', ' V', ' I']

# Additional Projections

## FantasyPros

In [ ]:
positions = ["qb","rb","wr","te",'dst']
#positions = ['dst']

fpdf_output = pd.DataFrame(columns=['Player', 'FPTS'])

for position in positions:
  URL = 'https://www.fantasypros.com/nfl/projections/%s.php' % position
  print(URL)
  response = requests.get(URL)

  soup = BeautifulSoup(response.content, 'html.parser')

  table = soup.find('table', {'id': 'data'})
  
  if position == 'qb':

    fpdf = pd.read_html(str(table))[0]

    fpdf.columns = fpdf.columns.droplevel(level=0)
    
    fpdf[['Player', 'Team']] = fpdf['Player'].str.rsplit(n=1, expand=True)

    fpdf['Player'] = fpdf['Player'].str.replace(r'[^\w]+', '', regex = True)

    fpdf.drop(fpdf.columns[[1,2,3,4,5,6,7,8,9,11]], axis=1, inplace=True)

  elif position == 'te':

    fpdf = pd.read_html(str(table))[0]

    fpdf.columns = fpdf.columns.droplevel(level=0)
    
    fpdf[['Player', 'Team']] = fpdf['Player'].str.rsplit(n=1, expand=True)

    fpdf['Player'] = fpdf['Player'].str.replace(r'[^\w]+', '', regex = True)

    fpdf.drop(fpdf.columns[[1,2,3,4,6]], axis=1, inplace=True)

  elif position == 'dst':

    fpdf = pd.read_html(str(table))[0]

    fpdf['Player'] = fpdf['Player'].str.replace(r'[^\w]+', '', regex = True)

    fpdf.drop(fpdf.columns[[1,2,3,4,5,6,7,8]], axis=1, inplace=True)
  
  else:

    fpdf = pd.read_html(str(table))[0]

    fpdf.columns = fpdf.columns.droplevel(level=0)
    
    fpdf[['Player', 'Team']] = fpdf['Player'].str.rsplit(n=1, expand=True)

    fpdf['Player'] = fpdf['Player'].str.replace(r'[^\w]+', '', regex = True)

    fpdf.drop(fpdf.columns[[1,2,3,4,5,6,7,9]], axis=1, inplace=True)

  fpdf_output = pd.concat([fpdf_output, fpdf])

fpdf_output


https://www.fantasypros.com/nfl/projections/qb.php
https://www.fantasypros.com/nfl/projections/rb.php
https://www.fantasypros.com/nfl/projections/wr.php
https://www.fantasypros.com/nfl/projections/te.php
https://www.fantasypros.com/nfl/projections/dst.php


,Player,FPTS
0,JoshAllen,25.2
1,PatrickMahomesII,23.4
2,JalenHurts,23.2
3,JustinHerbert,22.4
4,LamarJackson,21.6
...,...,...
21,LasVegasRaiders,5.6
22,WashingtonCommanders,5.4
23,AtlantaFalcons,5.3
24,LosAngelesRams,5.3


## CBS

In [ ]:
positions_cap = ["QB","RB","WR","TE","DST"]
#positions_cap = ["DST"]

cbsdf_output = pd.DataFrame(columns=['Player', 'FPTS'])

for position in positions_cap:
  URL = 'https://www.cbssports.com/fantasy/football/stats/%s/2022/tp/projections/ppr/' % position
  print(URL)
  response = requests.get(URL)

  soup = BeautifulSoup(response.content, 'html.parser')

  table = soup.find('table')

  cbsdf = pd.read_html(str(table))[0]

  cbsdf.columns = cbsdf.columns.droplevel(level=0)

  if position != 'DST':

    cbsdf['Player'] = cbsdf['Player'].str.split(n=0, expand=False)

    for i in range(len(cbsdf['Player'])):

      name_edit = cbsdf.loc[i, 'Player']
      del_index = name_edit.index(name_edit[-1])
      name_edit = name_edit[del_index +1:-2]
      
      name_edit = re.sub(r'[^\w\s]', '', ''.join(name_edit))

      cbsdf.loc[i, 'Player'] = name_edit

    if position == 'QB':

      print(cbsdf)

      cbsdf.drop(cbsdf.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13,15]], axis=1, inplace=True)

      print(cbsdf)

    elif position == 'RB':
    
      cbsdf.drop(cbsdf.columns[[1,2,3,4,5,6,7,8,9,10,11,12,14]], axis=1, inplace=True)

      print(cbsdf)

    elif position == 'WR':
    
      cbsdf.drop(cbsdf.columns[[1,2,3,4,5,6,7,8,9,10,11,12,14]], axis=1, inplace=True)

      print(cbsdf)

    elif position == 'TE':
    
      cbsdf.drop(cbsdf.columns[[1,2,3,4,5,6,7,8,10]], axis=1, inplace=True)

      print(cbsdf)

  else:
    cbsdf = cbsdf

    cbsdf['Team'] = cbsdf['Team'].str.replace(r'[^\w]+', '', regex = True)

    cbsdf.drop(cbsdf.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13,15]], axis=1, inplace=True)

    print(cbsdf)
  
  #cbsdf_output = pd.concat([cbsdf_output, cbsdf])

#cbsdf_output


https://www.cbssports.com/fantasy/football/stats/QB/2022/tp/projections/ppr/
              Player  gp  Games Played  att  Pass Attempts  \
0     PatrickMahomes                 1                38.5   
1          JoshAllen                 1                37.7   
2      JustinHerbert                 1                41.7   
3         JalenHurts                 1                30.8   
4          JoeBurrow                 1                36.8   
5       AaronRodgers                 1                34.5   
6      TuaTagovailoa                 1                37.9   
7       LamarJackson                 1                30.0   
8        KylerMurray                 1                36.2   
9      MarcusMariota                 1                28.8   
10       KirkCousins                 1                36.1   
11          TomBrady                 1                42.3   
12    TrevorLawrence                 1                37.3   
13         GenoSmith                 1                3

In [ ]:
#positions_cap = ["qb","rb","wr","te"]
positions_cap = ["TE"]

for position in positions_cap:
  URL = 'https://www.cbssports.com/fantasy/football/stats/%s/2022/tp/projections/ppr/' % position
  print(URL)
  response = requests.get(URL)

  soup = BeautifulSoup(response.content, 'html.parser')

  table = soup.find('table')

  cbsdf = pd.read_html(str(table))[0]

  cbsdf.columns = cbsdf.columns.droplevel(level=0)

  if position == 'QB':

    cbsdf['Player'] = cbsdf['Player'].str.split(n=0, expand=False)

    for i in range(len(cbsdf['Player'])):

      name_edit = cbsdf.loc[i, 'Player']
      del_index = name_edit.index(name_edit[-1])
      name_edit = name_edit[del_index +1:-2]
      
      name_edit = re.sub(r'[^\w\s]', '', ''.join(name_edit))

      cbsdf.loc[i, 'Player'] = name_edit

    cbsdf.drop(cbsdf.columns[[1,2,3,4,5,6,7,8,9,10,11,12,13,15]], axis=1, inplace=True)

  elif position == 'RB':
    
    cbsdf['Player'] = cbsdf['Player'].str.split(n=0, expand=False)

    for i in range(len(cbsdf['Player'])):

      name_edit = cbsdf.loc[i, 'Player']
      del_index = name_edit.index(name_edit[-1])
      name_edit = name_edit[del_index +1:-2]
      
      name_edit = re.sub(r'[^\w\s]', '', ''.join(name_edit))

      cbsdf.loc[i, 'Player'] = name_edit

    cbsdf.drop(cbsdf.columns[[1,2,3,4,5,6,7,8,9,10,11,12,14]], axis=1, inplace=True)

  elif position == 'WR':
    
    cbsdf['Player'] = cbsdf['Player'].str.split(n=0, expand=False)

    for i in range(len(cbsdf['Player'])):

      name_edit = cbsdf.loc[i, 'Player']
      del_index = name_edit.index(name_edit[-1])
      name_edit = name_edit[del_index +1:-2]
      
      name_edit = re.sub(r'[^\w\s]', '', ''.join(name_edit))

      cbsdf.loc[i, 'Player'] = name_edit

    cbsdf.drop(cbsdf.columns[[1,2,3,4,5,6,7,8,9,10,11,12,14]], axis=1, inplace=True)

  elif position == 'TE':
    
    cbsdf['Player'] = cbsdf['Player'].str.split(n=0, expand=False)

    for i in range(len(cbsdf['Player'])):

      name_edit = cbsdf.loc[i, 'Player']
      del_index = name_edit.index(name_edit[-1])
      name_edit = name_edit[del_index +1:-2]
      
      name_edit = re.sub(r'[^\w\s]', '', ''.join(name_edit))

      cbsdf.loc[i, 'Player'] = name_edit

    cbsdf.drop(cbsdf.columns[[1,2,3,4,5,6,7,8,10]], axis=1, inplace=True)

cbsdf


https://www.cbssports.com/fantasy/football/stats/TE/2022/tp/projections/ppr/


,Player,fpts Fantasy Points
0,MarkAndrews,17.6
1,TravisKelce,17.3
2,DarrenWaller,13.9
3,GeorgeKittle,11.9
4,ZachErtz,10.8
...,...,...
93,JakeTonges,0.8
94,KyleRudolph,0.8
95,DanArnold,0.7
96,EricTomlinson,0.7


Rostered players DF

In [ ]:
output_table = pd.DataFrame(columns = ["Player ID", "roster_id", "Player Name", "lookup name", "Position", "Team", "Sleeper Projected Points"])

#Master should probably be player ID's - I need to get list of all rostered starters, then associate owner name to them.

#cycle through rosters, store [roster_id, player_id]
i = 0
for roster in rosters_master:
  i = i+1
  print(i)
  holding_table = pd.DataFrame(columns = ["Player ID", "roster_id", "Player Name", "lookup name", "Position", "Team", "Sleeper Projected Points"])

  for starter in range(starting_roster_size):
    
    holding_list = []    

    playerid = roster["starters"][starter]

    holding_list.append(playerid)

    holding_list.append(roster["roster_id"])

    holding_list.append(players_master[playerid]["first_name"] + " " + players_master[playerid]["last_name"])

    holding_list.append(re.sub(r'[^\w]+','', players_master[playerid]["first_name"] + players_master[playerid]["last_name"]))
    
    holding_list.append(players_master[playerid]["position"])

    holding_list.append(players_master[playerid]["team"])

    #projected points?!

    if playerid in weekly_projections:

      holding_list.append(weekly_projections[playerid].get('pts_ppr',0))

    else:

      holding_list.append(0)

    holding_table.loc[len(holding_table)] = holding_list

#This adds the list to the end of the dataframe
  output_table = output_table.append(holding_table)

output_table

1
2
3
4
5
6
7
8
9
10
11
12


,Player ID,roster_id,Player Name,lookup name,Position,Team,Sleeper Projected Points
0,167,1,Tom Brady,TomBrady,QB,TB,22.24
1,6790,1,D'Andre Swift,DAndreSwift,RB,DET,15.45
2,5284,1,Jeff Wilson,JeffWilson,RB,SF,9.59
3,1426,1,DeAndre Hopkins,DeAndreHopkins,WR,ARI,14.59
4,2133,1,Davante Adams,DavanteAdams,WR,LV,20.72
...,...,...,...,...,...,...,...
4,4040,12,JuJu Smith-Schuster,JuJuSmithSchuster,WR,KC,11.94
5,2505,12,Darren Waller,DarrenWaller,TE,LV,10.26
6,6806,12,J.K. Dobbins,JKDobbins,RB,BAL,8.85
7,4033,12,David Njoku,DavidNjoku,TE,CLE,10.07


In [ ]:
totals_df = pd.DataFrame(columns = ["matchup id", "roster_id", "projected total"])

for matchup in matchups_master:

  list = []

  list.append(matchup["matchup_id"])
  list.append(matchup["roster_id"])

  list.append(output_table.loc[output_table['roster_id'] == matchup["roster_id"], 'Projected Points'].sum())

  totals_df.loc[len(totals_df)] = list

totals_df.sort_values(by=["projected total"], inplace=True, ascending=False)

totals_df

KeyError: ignored

# ARCHIVE

Roster master method

In [ ]:
#NOTE: in roster_library, L1 index is roster_id

roster_library = {}
owner_id_dict = {}

for owner in user_list_master:
  owner_id_dict[owner["user_id"]] = owner["display_name"]

for roster in rosters_master:
  #Create projected total container
  projected_total = []
  #Create roster_id index
  roster_library[roster["roster_id"]] = {}

  #Correlate usernames to roster IDs
  roster_library[roster["roster_id"]]["username"] = owner_id_dict[roster["owner_id"]]

  #Create Starters index
  roster_library[roster["roster_id"]]["starters"] = {}

  #Correlating starters stats (primary index is player id no)
  for starter in rosters_master[roster["roster_id"]-1]["starters"]:

    #Creating starters dict
    roster_library[roster["roster_id"]]["starters"][starter]={}

    #Associating first and last names to starters
    roster_library[roster["roster_id"]]["starters"][starter]["first name"] = players_master[starter]["first_name"]
    roster_library[roster["roster_id"]]["starters"][starter]["last name"] = players_master[starter]["last_name"]
    roster_library[roster["roster_id"]]["starters"][starter]["short name"] = re.sub(r'[^\w\s]','', players_master[starter]["first_name"] + players_master[starter]["last_name"])
    roster_library[roster["roster_id"]]["starters"][starter]["position"] = players_master[starter]["position"]

    #Associating projected points (NEED TO ADD other sources)
    roster_library[roster["roster_id"]]["starters"][starter]["sleeper proj pts"] = weekly_projections[starter]["pts_ppr"]
    projected_total.append(weekly_projections[starter]["pts_ppr"])
  
  #Projected Total
  roster_library[roster["roster_id"]]["projected total"] = sum(projected_total)

roster_library

Player master method

In [ ]:
players = players_master
owner_id_dict = {}

for owner in user_list_master:
  owner_id_dict[owner["user_id"]] = owner["display_name"]

for player in list(players.keys()):
  for i in range(12):
    if player in rosters_master[i]["starters"]:
      players[player]["owner"] = owner_id_dict[roster["owner_id"]]

players

Matchup Library

In [ ]:
#matchups[matchup id][roster id]

matchups = {}

for i in range(1,6):
  matchup_holder = {}
  for matchup in matchups_master:
    if matchup["matchup_id"] == i:
      matchup_holder[matchup["roster_id"]] = 0
  matchups[i] = matchup_holder

matchups

Safekeeping

In [ ]:
matchups = {}

for i in range(5):
  matchup_holder = {}
  for matchup in matchups_master:
    if matchup["matchup_id"] == i+1:
      matchup_holder[matchup["roster_id"]] = 0
  matchups[i+1] = matchup_holder

print(matchups)

Tests

In [ ]:
rosters_master[0]["starters"]

In [ ]:
matchups_master

In [ ]:
league.get_rosters()

In [ ]:
league.get_users()